## SW分类数据
导出所有行业筛选出的上升标记的公司列表

In [1]:
import sys
sys.path.append("..")
from roeProcessTools import *

In [2]:
import os

In [3]:
# dirPath = "/Users/zch/Desktop/internship/SWData/"
# outPath = dirPath+"/cleanData/"

In [4]:
# 导出数据
with os.scandir(dirPath+"rawData") as it:
    for entry in it:
        if not entry.name.startswith("SW"):
            continue
        # print(entry.path)
        df = pd.read_excel(entry.path)
        df = clean(df, startYear=2000, endYear=2021, minNum=32, winsorizeBool=True)
        df.to_excel(outPath+entry.name, index=False)
        # marked[entry.name[2:]] = mark(df, n=8, center=False, gaussian=True, order=4, threshold=0.8, highPct=75, increaseBool=False)

In [5]:
marked = {}
with os.scandir(outPath) as it:
    for entry in it:
        if not entry.name.startswith("SW"):
            continue
        df = pd.read_excel(entry.path)
        marked[entry.name[2:]] = mark(df, n=8, center=False, gaussian=True, order=4, threshold=0.8, highPct=75, increaseBool=True)

In [28]:
count = 0
for k, v in marked.items():
    if v.empty:
        print("empty", k)
        continue
    # v.to_excel(dirPath+"selected/"+k,index=False)
    count+=len(v)
    print(k, len(v))
print(count)

empty 银行.xlsx
食品饮料.xlsx 14
电气设备.xlsx 16
商业贸易.xlsx 13
轻工制造.xlsx 12
机械设备.xlsx 36
农林牧渔.xlsx 16
综合.xlsx 5
国防军工.xlsx 10
纺织服装.xlsx 3
建筑装饰.xlsx 8
电子.xlsx 24
建筑材料.xlsx 9
empty 休闲服务.xlsx
汽车.xlsx 15
有色金属.xlsx 15
交通运输.xlsx 6
计算机.xlsx 21
采掘.xlsx 6
非银金融.xlsx 5
家用电器.xlsx 7
钢铁.xlsx 3
传媒.xlsx 8
房地产.xlsx 18
公用事业.xlsx 15
医药生物.xlsx 19
化工.xlsx 8
通信.xlsx 7
319


In [24]:
for k,v in marked.items():
    if v.empty:
        print(k)
        continue
    v["industry"] = k[:-5]

银行.xlsx
休闲服务.xlsx


In [26]:
temp = pd.concat(marked.values(), axis=0)

In [30]:
# 导出全部选择的结果
# temp.to_excel(dirPath+"selected/total.xlsx", index=False)

In [34]:
# 导出全部选择公司的roe数据，方便绘图
dfs = []
with os.scandir(outPath) as it:
    for entry in it:
        if not entry.name.startswith("SW"):
            continue
        # temp = pd.read_excel(entry.path)
        # temp["industry"] = entry.name[2:-5]
        dfs.append(pd.read_excel(entry.path))
totalDf = pd.concat(dfs, axis=0)

In [37]:
codes = temp.code.values
len(codes)

319

In [52]:
totalDf = pd.concat([totalDf.iloc[:, :2],totalDf.iloc[:, 2:].sort_index(axis=1)], axis=1)

In [53]:
temp1 = totalDf[totalDf["code"].apply(lambda x: x in codes)]

In [55]:
# temp1.to_excel(outPath+"selected.xlsx", index=False)

# test part 

In [7]:
df = pd.read_excel(dirPath+"rawData/"+"SW汽车.xlsx")

In [8]:
# codes = df.code.values
# marks = []
# startTime = []
# for code in codes:
#     company, y = getCompanyByCode(code, df)
#     yRolling = getYRolling(y, n=8, gaussian=True, center=False)
#     yRolling.dropna(inplace=True)
#     maxIdx, minIdx = getExtreme(yRolling, order=4)
#     maximum, minimum = yRolling[maxIdx], yRolling[minIdx]
#     marks.append(markCompany(yRolling, maximum, minimum))
#     startTime.append(minimum.index[-1])
# temp = pd.DataFrame(data={"code": codes, "name": df.name.values, 
#                     "mark": marks,  "startTime": startTime})

In [9]:
# test 去除近年没有数据的行
# 000927 
# 中国铁物

In [10]:
temp = df.iloc[20:21,:]

In [13]:
columns = ["code", "name"]
for year in range(2000, 2021):
    for month in range(3, 13, 3):  # 3 6 9 12
        columns.append("%d-%02d" % (year, month))
df.columns = columns 

In [15]:
# 去除空行
df = df[~(df["name"].isnull())]
# 去除ST
df = df[~df["name"].str.contains("ST")]
# 去除全为空的列，即没有数据的年份
df.dropna(how="all", axis=1, inplace=True)

In [20]:
# 去除近年没有数据的公司
yearColumn = df.columns[-1]
df = df[~df[yearColumn].isnull()]

In [21]:
# 去除不连续的公司，空缺roe不超过1
df = df[df.apply(lambda x:isContinuous(x), axis=1)]

In [21]:
data = pd.read_excel(dirPath+"rawData/SW电气设备.xlsx")

In [23]:
# 更改列名
columns = ["code", "name"]
for year in range(2000, 2021):
    for month in range(3, 13, 3):  # 3 6 9 12
        columns.append("%d-%02d" % (year, month))
data.columns = columns 

In [25]:
len(data)

224

In [26]:
# 去除空行
data = data[~(data["name"].isnull())]
len(data)

224

In [27]:
# 去除ST
data = data[~data["name"].str.contains("ST")]
len(data)

212

In [28]:
# 去除全为空的列，即没有数据的年份
data.dropna(how="all", axis=1, inplace=True)
len(data)

212

In [32]:
# 去除近年没有数据的公司
c1 = data.columns[-1]
c2 = data.columns[-2]
data1 = data[~((data[c1].isnull())&(data[c2].isnull()))] 
# data1 = data[~data[yearColumn].isnull()] 
len(data1)

212

In [17]:
# 去除不连续的公司，空缺roe不超过1
data = data[data.apply(lambda x:isContinuous(x), axis=1)]

In [19]:
df1

,code,name,2001-09,2001-12,2002-03,2002-06,2002-09,2002-12,2003-03,2003-06,...,2018-09,2018-12,2019-03,2019-06,2019-09,2019-12,2020-03,2020-06,2020-09,2020-12
116,300617.SZ,安靠智电,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.9886,1.7921,2.501,1.92,1.1877,2.1659,3.2737,4.7269,7.0499,0.569
